**Импорты и константы**

In [1]:
!pip install -U trl datasets peft bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlin

In [2]:
!pip install transformers==4.49.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 77.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.1
    Uninstalling transformers-4.51.1:
      Successfully uninstalled transformers-4.51.1


In [3]:
from datasets import load_dataset, DatasetDict
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from accelerate import Accelerator, PartialState
from accelerate.utils import write_basic_config
from peft import LoraConfig,prepare_model_for_kbit_training
from torch import nn
import os, torch

# Выключаем логгирование
os.environ['WANDB_DISABLED']="true"

data_path ="/kaggle/input/misis-final-dataset"
model_name = "yandex/YandexGPT-5-Lite-8B-pretrain"
output_directory = "/kaggle/working/"

2025-04-26 19:14:29.961054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745694870.238815      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745694870.308097      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**Функции**

In [4]:
def formatting_prompts_func(data, last_mes_amount=5):
    retrieved = "\n".join(
        [f"Пользователь1: {x['instruction']}\nПользователь2: {x['output']}" for x in data['retrieved_context'][-last_mes_amount:]])
    if retrieved:
        prompt = f"{retrieved}\nПользователь1: {data['instruction']}\nПользователь2: "
    else:
        prompt = f"Пользователь1: {data['instruction']}\nПользователь2: "
    # Можно ипользовать просто instruction_template="Пользователь2:" тк ищет последнее вхождение
    return {'prompt': prompt, 'completion': data['output']}

**Загружаем датасет**

In [5]:
data = load_dataset(data_path, split="train").map(formatting_prompts_func)
data = data.remove_columns(['instruction', 'output', 'retrieved_context'])

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/7317 [00:00<?, ? examples/s]

**Подгружаем модель и токенайзер**

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map='auto',
    quantization_config=bnb_config,
    use_cache=False,
    #max_memory={0:"14GiB", 1:"14GiB", "cpu":"10GiB"}
)
model = prepare_model_for_kbit_training(model)
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,
                                          padding_side="left", # Обрезаем начало, чтобы сохранять в контексте диалога последние сообщения
                                          add_eos_token=True,add_bos_token=True,
                                          use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/542 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/2.57M [00:00<?, ?B/s]

In [8]:
print(*(x for x in model.named_modules()))

('', LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(129024, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-06)
      )
    )
    (norm): LlamaRMSNorm

In [9]:
# instruction_template = "### PROMPT:"
# response_template = "### OUTPUT:"
# collator = DataCollatorForCompletionOnlyLM(instruction_template=instruction_template, response_template=response_template, 
#                                            tokenizer=tokenizer, mlm=False) # mlm=False тк обучаем генеративную модель, а не masked

**Fine-Tuning**

In [10]:
peft_config = LoraConfig(
            r=8,  # Чем больше R тем больше параметров
            lora_alpha=16,  # Величина матрицы
            target_modules=["q_proj", "k_proj", "v_proj"],  # Дообучаем Q, K и V механизма внимания
            lora_dropout=0.01,  # Чтоб не переобучиться
            bias="all",  # Меняем байес во всех слоях
            task_type="CAUSAL_LM"  # Обучаем генеративную модель
        )

training_args=SFTConfig(
    label_names=["labels"],
    output_dir=output_directory,
    
    per_device_train_batch_size=1,  # Размер батча для тренировки на одном устройстве (GPU/CPU)
    per_device_eval_batch_size=1,  # Размер батча для валидации на одном устройстве

    gradient_checkpointing = False,
    gradient_checkpointing_kwargs = {"use_reentrant": False}, #must be false for DDP

    gradient_accumulation_steps=1,  # Сколько раз будет накапливаться градиент перед шагом обновления
    num_train_epochs=2.0,  # Количество эпох для тренировки модели

    # Оптимизация
    learning_rate=2e-5,  # lr
    max_grad_norm=1.0,  # Ограничение на норму градиента

    # Логирование и сохранение
    logging_strategy="steps",  # Логирование через несколько шагов
    logging_steps=500,  # Логирование каждые 500 шагов
    save_strategy="steps",  # Сохранение модели после каждого шага
    save_steps=1000,  # Каждые 1000 шагов
    save_total_limit=3,  # Не более 3 последних чекпоинтов
    save_safetensors=True,  # Используем формат .safetensors

    # Использование 16-битных вычислений для ускорения
    fp16=True,  # Испоьзуем 16-битные числа
    bf16=False,  # Обучаем не на A100(

    seed=42,

    remove_unused_columns=True,  # Убираем ненужные колонки
    report_to=None,  # Отключаем логгирование
    push_to_hub=False,  # Не пушим на hf

    # Параметры для загрузчика DataLoader
    ddp_find_unused_parameters=False,  # Для распределенного обучения
    dataloader_pin_memory=False,  # Может вызывать конфликты с device_map="auto
    skip_memory_metrics=True,  # Отключаем вычисление метрик памяти, чтобы уменьшить нагрузку на систему
    disable_tqdm=False,  # Прогресс-бар
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [11]:
trainer = SFTTrainer(model=model,
                    peft_config=peft_config,
                    train_dataset=data,
                    #data_collator=collator,
                    args=training_args,
)

Converting train dataset to ChatML:   0%|          | 0/7317 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/7317 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7317 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:597: UserWarning: Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.
  warnings.warn(


Truncating train dataset:   0%|          | 0/7317 [00:00<?, ? examples/s]

In [12]:
trainer.args

SFTConfig(output_dir='/kaggle/working/', overwrite_output_dir=False, do_train=False, do_eval=False, do_predict=False, eval_strategy=<IntervalStrategy.NO: 'no'>, prediction_loss_only=False, per_device_train_batch_size=1, per_device_eval_batch_size=1, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, eval_delay=0, torch_empty_cache_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, lr_scheduler_kwargs={}, warmup_ratio=0.0, warmup_steps=0, log_level='passive', log_level_replica='warning', log_on_each_node=True, logging_dir='/kaggle/working/runs/Apr26_19-17-39_90f0a8094ddb', logging_strategy=<IntervalStrategy.STEPS: 'steps'>, logging_first_step=False, logging_steps=500, logging_nan_inf_filter=True, save_strategy=<SaveStrategy.STEPS: 'steps'>, save_steps=1000, s

In [13]:
print("Распределение модели:", model.hf_device_map)

Распределение модели: {'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 0, 'model.layers.7': 0, 'model.layers.8': 1, 'model.layers.9': 1, 'model.layers.10': 1, 'model.layers.11': 1, 'model.layers.12': 1, 'model.layers.13': 1, 'model.layers.14': 1, 'model.layers.15': 1, 'model.layers.16': 1, 'model.layers.17': 1, 'model.layers.18': 1, 'model.layers.19': 1, 'model.layers.20': 1, 'model.layers.21': 1, 'model.layers.22': 1, 'model.layers.23': 1, 'model.layers.24': 1, 'model.layers.25': 1, 'model.layers.26': 1, 'model.layers.27': 1, 'model.layers.28': 1, 'model.layers.29': 1, 'model.layers.30': 1, 'model.layers.31': 1, 'model.norm': 1, 'model.rotary_emb': 1, 'lm_head': 1}


In [14]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
500,3.896400
1000,3.713300
1500,3.609100
2000,3.571600
2500,3.604400
3000,3.609500
3500,3.614200
4000,3.487500
4500,3.525800
5000,3.526000


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/

TrainOutput(global_step=14634, training_loss=3.490672292027689, metrics={'train_runtime': 30703.8827, 'train_samples_per_second': 0.477, 'train_steps_per_second': 0.477, 'total_flos': 1.5818679076960666e+17, 'train_loss': 3.490672292027689})

In [15]:
trainer.model.save_pretrained(os.path.join(output_directory, f"lora_model"))